# Firebolt Core Examples

This notebook demonstrates using the Firebolt Python SDK with Firebolt Core, a Docker-based version of Firebolt for local or remote use.

## Prerequisites

Firebolt Core can be run locally using Docker. Refer to the [official docs](https://docs.firebolt.io/firebolt-core/firebolt-core-get-started) on how to run it.

For this notebook, we'll assume Firebolt Core is running locally on the default port (3473).

## Connection Types

There are two ways to connect to Firebolt Core:

1. **Synchronous connection** - using the standard DB-API 2.0 interface
2. **Asynchronous connection** - using the async equivalent API

Both methods use the `FireboltCore` authentication class, which doesn't require actual credentials.

### Required imports

In [ ]:
from firebolt.db import connect
from firebolt.client.auth import FireboltCore

### Connection settings

In [ ]:
# For Firebolt Core, we don't need credentials
# but we can specify the database and host if needed
database = "firebolt"  # Default database name
url = "http://localhost:3473"  # Default URL

### Connecting to a database and creating cursor

In [ ]:
# Create a FireboltCore auth object - no credentials needed
auth = FireboltCore()

# Connect to Firebolt Core
connection = connect(auth=auth, database=database, url=url)

# Create a cursor
cursor = connection.cursor()

### Executing a query

In [ ]:
# Create a test table
cursor.execute(
    "CREATE TABLE IF NOT EXISTS example_table (id INT, name TEXT, value FLOAT, created_at TIMESTAMP)"
)

# Insert some test data
cursor.execute(
    "INSERT INTO example_table VALUES "
    "(1, 'Item 1', 10.5, '2023-01-01 12:00:00'), "
    "(2, 'Item 2', 20.75, '2023-01-02 14:30:00'), "
    "(3, 'Item 3', 15.25, '2023-01-03 09:45:00')"
)

# Execute a simple test query
cursor.execute("SELECT 1")

# Fetch and display results
result = cursor.fetchall()
print(f"Query result: {result}")

# Get column names
print(f"Column names: {cursor.description[0][0]}")

# Show connection parameters (filtered for Firebolt Core)
print(f"Connection URL: {connection.engine_url}")

### Parameterized query

In [ ]:
# Execute with a single parameter set
cursor.execute(
    "INSERT INTO example_table VALUES (?, ?, ?, ?)",
    (4, "Parameter Example", 30.5, "2023-01-04 10:00:00"),
)

# Execute with multiple parameter sets
cursor.executemany(
    "INSERT INTO example_table VALUES (?, ?, ?, ?)",
    [
        (5, "Multi Param 1", 25.5, "2023-01-05 11:00:00"),
        (6, "Multi Param 2", 35.75, "2023-01-06 12:00:00"),
    ],
)

### Getting query description, rowcount

In [ ]:
cursor.execute("SELECT * FROM example_table")
print("Description: ", cursor.description)
print("Rowcount: ", cursor.rowcount)

### Fetch query results

In [ ]:
# Re-run the query to reset the cursor position
cursor.execute("SELECT * FROM example_table")

# Different fetch methods
print("fetchone():", cursor.fetchone())
print("fetchmany(2):", cursor.fetchmany(2))
print("fetchall():", cursor.fetchall())

### Closing the connection

In [ ]:
connection.close()

### Async interface
**NOTE**: In order to make async examples work in jupyter, you would need to install [trio-jupyter](https://github.com/mehaase/trio-jupyter) library and select **Python 3 Trio** kernel

#### Required imports

In [ ]:
from firebolt.async_db import connect as async_connect
from firebolt.client.auth import FireboltCore

In [ ]:
auth = FireboltCore()
# Connect to Firebolt Core asynchronously
connection = await async_connect(auth=auth, database="firebolt")

# Create a cursor
cursor = connection.cursor()

In [ ]:
await cursor.execute("SELECT 2")

# Fetch and display results
result = await cursor.fetchall()
print(f"Async query result: {result}")

# Get column names
print(f"Column names: {cursor.description[0][0]}")

In [ ]:
# Close connection when done
await connection.aclose()